In [ ]:
import pathlib
import pandas as pd

datadir = pathlib.Path.cwd().parent / "data"

pd_hotel       = pd.read_parquet(datadir / "hotel.parquet")
pd_customer    = pd.read_parquet(datadir / "customer.parquet")
pd_reservation = pd.read_parquet(datadir / "reservation.parquet")

# 6章 集約
## 6-1 データ全体の集約と代表的な集約関数
### Q：予約履歴の各種集計値の算出


#### Awesome

In [ ]:
(
    pd_reservation
    # （1）キャンセル済みではない予約の抽出
    .query("status != 'canceled'")
    # （2）集計値の計算
    .agg(
        reservation_cnt=("reservation_id", "count"), # カウント
        sales=("total_price", "sum"),                # 総和
        mean_sales=("total_price", "mean"),          # 平均値
        min_sales=("total_price", "min"),            # 最小値
        max_sales=("total_price", "max"),            # 最大値
        var_sales=("total_price", "var"),            # 不偏分散
        std_sales=("total_price", "std"),            # 不偏標準偏差
    )
)

### Q：予約顧客のユニークカウントの算出


#### Awesome

In [ ]:
(
    pd_reservation
    # （1）キャンセル済みではない予約の抽出
    .query("status != 'canceled'")
    # （2）ユニークカウントの計算
    .customer_id.nunique()
)

### Q：予約単価の中央値およびパーセンタイル値の算出


#### Awesome

In [ ]:
(
    pd_reservation
    # （1）キャンセル済みではない予約の抽出
    .query("status != 'canceled'")
    # （2）中央値、パーセンタイル値の計算
    .agg(
        median_sales=("total_price", "median"),
        p25_sales=("total_price", lambda s: s.quantile(0.25)),
        p75_sales=("total_price", lambda s: s.quantile(0.75)),
    )
)

### Q：ホテルごとの宿泊人数の最頻値の算出


#### Awesome

In [ ]:
(
    pd_reservation
    # （1）キャンセル済みではない予約の抽出
    .query("status != 'canceled'")
    # （2）グループごとに最頻値を取得
    .groupby("hotel_id").agg(mode_people_num=("people_num", lambda s: s.mode().iloc[0]))
)

## 6-2 グループごとの集約
### Q：ホテルごとの売上の集計


#### Awesome

In [ ]:
(
    pd_reservation
    # （1）キャンセル済みではない予約の抽出
    .query("status != 'canceled'")
    # （2）hotel_idの値ごとにtotal_priceの総和を計算
    .groupby("hotel_id").total_price.sum()
)

### Q：ホテルごと・顧客ごとの予約数の集計


#### Awesome

In [ ]:
(
    pd_reservation
    # （1）キャンセル済みではない予約の抽出
    .query("status != 'canceled'")
    # （2）hotel_idとcustomer_idの値の組ごとにデータ数をカウント
    .groupby(["hotel_id", "customer_id"]).size()
)

## 6-3 数値の区間ごとの集約
### Q：等間隔の価格帯ごとにホテル数を集計


#### Awesome

In [ ]:
import numpy as np

(
    pd_hotel
    # （1）unit_priceを数値区間に丸めた列を作成
    .assign(unit_price_range=lambda df:
        (np.floor(df.unit_price / 5000) * 5000).astype(int))
    # （2）unit_price_rangeの値ごとにデータ数をカウント
    .groupby("unit_price_range").size()
)

### Q：非等間隔の価格帯ごとにホテル数を集計


#### Not Awesome

In [ ]:
# （1）unit_priceを数値区間に丸めるための関数を定義
def calc_unit_price_range(x):
    if x < 5000:
        return "1: 0~4999"
    elif x < 10000:
        return "2: 5000~9999"
    elif x < 20000:
        return "3: 10000~19999"
    elif x < 30000:
        return "4: 20000~29999"
    else:
        return "5: 30000~"

(
    pd_hotel
    # （2）unit_priceを数値区間に丸めた列を作成
    .assign(unit_price_range=lambda df: df.unit_price.apply(calc_unit_price_range))
    # （3）unit_price_rangeの値ごとにデータ数をカウント
    .groupby("unit_price_range").size()
)

#### Not Awesome

In [ ]:
import numpy as np

(
    pd_hotel
    # （1）unit_priceを数値区間に丸めた列を作成
    .assign(unit_price_range=lambda df:
        np.where(df.unit_price < 5000, "1: 0~4999",
        np.where(df.unit_price < 10000, "2: 5000~9999",
        np.where(df.unit_price < 20000, "3: 10000~19999",
        np.where(df.unit_price < 30000, "4: 20000~29999",
            "5: 30000~")))))
    # （2）unit_price_rangeの値ごとにデータ数をカウント
    .groupby("unit_price_range").size()
)

#### Awesome

In [ ]:
import numpy as np

(
    pd_hotel
    # （1）unit_priceを数値区間に丸めた列を作成
    .assign(unit_price_range=lambda df:
        np.where(df.unit_price < 5000, 0,
        np.where(df.unit_price < 10000, 5000,
        np.where(df.unit_price < 20000, 10000,
        np.where(df.unit_price < 30000, 20000,
            30000)))))
    # （2）unit_price_rangeの値ごとにデータ数をカウント
    .groupby("unit_price_range").size()
)

#### Awesome

In [ ]:
import numpy as np

(
    pd_hotel
    .groupby(
        # cut関数で定義した数値区間ごとにデータ数をカウント
        pd.cut(pd_hotel.unit_price, [-np.inf, 5000, 10000, 20000, 30000, np.inf],
            right=False)
    ).size()
)

## 6-4 時間の区間ごとの集約
### Q：月ごとの売上の集計


#### Not Awesome

In [ ]:
(
    pd_reservation
    # （1）キャンセル済みではない予約の抽出
    .query("status != 'canceled'")
    # （2）checkout_dateの年月部分を取り出した文字列を作成
    .assign(month=lambda df: df.checkout_date.dt.strftime("%Y-%m"))
    # （3）monthの値ごとにtotal_priceの総和を計算
    .groupby("month").total_price.sum()
)

#### Awesome

In [ ]:
(
    pd_reservation
    # （1）キャンセル済みではない予約の抽出
    .query("status != 'canceled'")
    # （2）checkout_dateを月ごとのPeriod型に変換
    .assign(month=lambda df: df.checkout_date.dt.to_period("M"))
    # （3）monthの値ごとにtotal_priceの総和を計算
    .groupby("month").total_price.sum()
)

## 6-5 条件を満たす行の存在判定
### Q：チェックインの7日以内に予約をキャンセルしたことがある顧客の判定


#### Awesome

In [ ]:
(
    pd_reservation
    #（1）キャンセル済み、かつcanceled_atがcheckin_dateの7日以内、の判定結果を表す列を作成
    .assign(is_canceled_within_7days_to_checkin=lambda df:
        (df.status == "canceled") & ((df.checkin_date - df.canceled_at).dt.days <= 7)
    )
    #（2）customer_idの値ごとに、is_canceled_within_7days_to_checkinがTrueとなるデータが
    #    1件以上存在するかどうかを集計
    .groupby("customer_id").is_canceled_within_7days_to_checkin.max()
)

## 6-6 条件を満たす行のみの集約
### Q：顧客ごとの売上とキャンセル率の算出


#### Awesome

In [ ]:
import numpy as np

(
    pd_reservation
    #（1）未キャンセルデータやキャンセル済みデータに限定した集計用の列を作成
    .assign(
        cancel_cnt=lambda df: np.where(df.status == "canceled", 1, 0),
        total_price_without_canceled=lambda df:
            np.where(df.status == "reserved", df.total_price, 0)
    )
    #（2）customer_idの値ごとにカウント、および（1）で作成した列の総和を計算
    .groupby("customer_id")
    .agg({
        "reservation_id": "count",
        "cancel_cnt": "sum",
        "total_price_without_canceled": "sum"
    })
    #（3）cancel_rateを計算
    .assign(cancel_rate=lambda df: df.cancel_cnt / df.reservation_id)
)